In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from tensorflow.python.keras.layers import Layer
from keras import backend as K
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utils import LRN2D
import utils
import sys

%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=sys.maxsize)

In [2]:
myInput = Input(shape=(96, 96, 3))

x = ZeroPadding2D(padding=(3, 3), input_shape=(96, 96, 3))(myInput)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)
x = Lambda(LRN2D, name='lrn_1')(x)
x = Conv2D(64, (1, 1), name='conv2')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = Conv2D(192, (3, 3), name='conv3')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = Lambda(LRN2D, name='lrn_2')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

# Inception3a
inception_3a_3x3 = Conv2D(96, (1, 1), name='inception_3a_3x3_conv1')(x)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn1')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)
inception_3a_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3)
inception_3a_3x3 = Conv2D(128, (3, 3), name='inception_3a_3x3_conv2')(inception_3a_3x3)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn2')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)

inception_3a_5x5 = Conv2D(16, (1, 1), name='inception_3a_5x5_conv1')(x)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn1')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)
inception_3a_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5)
inception_3a_5x5 = Conv2D(32, (5, 5), name='inception_3a_5x5_conv2')(inception_3a_5x5)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn2')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)

inception_3a_pool = MaxPooling2D(pool_size=3, strides=2)(x)
inception_3a_pool = Conv2D(32, (1, 1), name='inception_3a_pool_conv')(inception_3a_pool)
inception_3a_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_pool_bn')(inception_3a_pool)
inception_3a_pool = Activation('relu')(inception_3a_pool)
inception_3a_pool = ZeroPadding2D(padding=((3, 4), (3, 4)))(inception_3a_pool)

inception_3a_1x1 = Conv2D(64, (1, 1), name='inception_3a_1x1_conv')(x)
inception_3a_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_1x1_bn')(inception_3a_1x1)
inception_3a_1x1 = Activation('relu')(inception_3a_1x1)

inception_3a = concatenate([inception_3a_3x3, inception_3a_5x5, inception_3a_pool, inception_3a_1x1], axis=3)

# Inception3b
inception_3b_3x3 = Conv2D(96, (1, 1), name='inception_3b_3x3_conv1')(inception_3a)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn1')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)
inception_3b_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3)
inception_3b_3x3 = Conv2D(128, (3, 3), name='inception_3b_3x3_conv2')(inception_3b_3x3)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn2')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)

inception_3b_5x5 = Conv2D(32, (1, 1), name='inception_3b_5x5_conv1')(inception_3a)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn1')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)
inception_3b_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5)
inception_3b_5x5 = Conv2D(64, (5, 5), name='inception_3b_5x5_conv2')(inception_3b_5x5)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn2')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)

inception_3b_pool = Lambda(lambda x: x**2, name='power2_3b')(inception_3a)
inception_3b_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: x*9, name='mult9_3b')(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_3b')(inception_3b_pool)
inception_3b_pool = Conv2D(64, (1, 1), name='inception_3b_pool_conv')(inception_3b_pool)
inception_3b_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_pool_bn')(inception_3b_pool)
inception_3b_pool = Activation('relu')(inception_3b_pool)
inception_3b_pool = ZeroPadding2D(padding=(4, 4))(inception_3b_pool)

inception_3b_1x1 = Conv2D(64, (1, 1), name='inception_3b_1x1_conv')(inception_3a)
inception_3b_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_1x1_bn')(inception_3b_1x1)
inception_3b_1x1 = Activation('relu')(inception_3b_1x1)

inception_3b = concatenate([inception_3b_3x3, inception_3b_5x5, inception_3b_pool, inception_3b_1x1], axis=3)

# Inception3c
inception_3c_3x3 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_3x3',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))

inception_3c_5x5 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))

inception_3c_pool = MaxPooling2D(pool_size=3, strides=2)(inception_3b)
inception_3c_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_3c_pool)

inception_3c = concatenate([inception_3c_3x3, inception_3c_5x5, inception_3c_pool], axis=3)

#inception 4a
inception_4a_3x3 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=192,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_4a_5x5 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(1, 1),
                                   padding=(2, 2))

inception_4a_pool = Lambda(lambda x: x**2, name='power2_4a')(inception_3c)
inception_4a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: x*9, name='mult9_4a')(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_4a')(inception_4a_pool)
inception_4a_pool = utils.conv2d_bn(inception_4a_pool,
                                   layer='inception_4a_pool',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   padding=(2, 2))
inception_4a_1x1 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_4a = concatenate([inception_4a_3x3, inception_4a_5x5, inception_4a_pool, inception_4a_1x1], axis=3)

#inception4e
inception_4e_3x3 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_3x3',
                                   cv1_out=160,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))
inception_4e_5x5 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_5x5',
                                   cv1_out=64,
                                   cv1_filter=(1, 1),
                                   cv2_out=128,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))
inception_4e_pool = MaxPooling2D(pool_size=3, strides=2)(inception_4a)
inception_4e_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_4e_pool)

inception_4e = concatenate([inception_4e_3x3, inception_4e_5x5, inception_4e_pool], axis=3)

#inception5a
inception_5a_3x3 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))

inception_5a_pool = Lambda(lambda x: x**2, name='power2_5a')(inception_4e)
inception_5a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: x*9, name='mult9_5a')(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_5a')(inception_5a_pool)
inception_5a_pool = utils.conv2d_bn(inception_5a_pool,
                                   layer='inception_5a_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   padding=(1, 1))
inception_5a_1x1 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))

inception_5a = concatenate([inception_5a_3x3, inception_5a_pool, inception_5a_1x1], axis=3)

#inception_5b
inception_5b_3x3 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_5b_pool = MaxPooling2D(pool_size=3, strides=2)(inception_5a)
inception_5b_pool = utils.conv2d_bn(inception_5b_pool,
                                   layer='inception_5b_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1))
inception_5b_pool = ZeroPadding2D(padding=(1, 1))(inception_5b_pool)

inception_5b_1x1 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_5b = concatenate([inception_5b_3x3, inception_5b_pool, inception_5b_1x1], axis=3)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)

# Final Model
model = Model(inputs=[myInput], outputs=norm_layer)

In [3]:
# Load weights from csv files (which was exported from Openface torch model)
weights = utils.weights
weights_dict = utils.load_weights()

# Set layer weights of the model
for name in weights:
  if model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])
  elif model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])

In [4]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding


In [5]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        

        print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance < 0.68:
        return str(name)
    else:
        return None

In [6]:
import glob

def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("images/*"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = image_to_embedding(image_file, model)

    return input_embeddings

def recognize_faces_in_cam(input_embeddings):
    
    cv2.namedWindow("Face Recognizer")
    vc = cv2.VideoCapture(0)
   
    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        height, width, channels = frame.shape
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h

            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)

            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", img)

        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

In [7]:
cam = cv2.VideoCapture(0)

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

count = 0
while(True):
    ret, img = cam.read()
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(img, 1.3, 5)
    for (x,y,w,h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
        count += 1
        # Save the captured image into the datasets folder
        cv2.imwrite("images/User_" + str(count) + ".jpg", img[y1:y2,x1:x2])
        cv2.imshow('image', img)
    k = cv2.waitKey(200) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= 10: # Take 30 face sample and stop video
         break
cam.release()
cv2.destroyAllWindows()

In [8]:
input_embeddings = create_input_image_embeddings()
recognize_faces_in_cam(input_embeddings)

Euclidean distance from Kunal is 1.1605928
Euclidean distance from mohit is 0.59028333
Euclidean distance from mohit_1 is 0.80546576
Euclidean distance from Kunal is 1.143046
Euclidean distance from mohit is 0.6212516
Euclidean distance from mohit_1 is 0.8263281
Euclidean distance from Kunal is 1.0879235
Euclidean distance from mohit is 0.8266777
Euclidean distance from mohit_1 is 1.0117753
Euclidean distance from Kunal is 1.2138455
Euclidean distance from mohit is 0.6306341
Euclidean distance from mohit_1 is 0.85879874
Euclidean distance from Kunal is 1.244595
Euclidean distance from mohit is 0.6382149
Euclidean distance from mohit_1 is 0.8768616
Euclidean distance from Kunal is 1.2251772
Euclidean distance from mohit is 0.5187064
Euclidean distance from mohit_1 is 0.7791497
Euclidean distance from Kunal is 1.2777052
Euclidean distance from mohit is 0.6474183
Euclidean distance from mohit_1 is 0.8855808
Euclidean distance from Kunal is 1.2828254
Euclidean distance from mohit is 0.6632

Euclidean distance from Kunal is 1.2430093
Euclidean distance from mohit is 0.5436847
Euclidean distance from mohit_1 is 0.78235525
Euclidean distance from Kunal is 1.2373706
Euclidean distance from mohit is 0.70107067
Euclidean distance from mohit_1 is 0.93112457
Euclidean distance from Kunal is 1.201886
Euclidean distance from mohit is 0.69120425
Euclidean distance from mohit_1 is 0.9149582
Euclidean distance from Kunal is 1.2693249
Euclidean distance from mohit is 0.60167646
Euclidean distance from mohit_1 is 0.83295834
Euclidean distance from Kunal is 1.1950771
Euclidean distance from mohit is 0.58629286
Euclidean distance from mohit_1 is 0.8321441
Euclidean distance from Kunal is 1.2138547
Euclidean distance from mohit is 0.7552334
Euclidean distance from mohit_1 is 0.9123674
Euclidean distance from Kunal is 1.389351
Euclidean distance from mohit is 0.577064
Euclidean distance from mohit_1 is 0.5849055
Euclidean distance from Kunal is 1.2978768
Euclidean distance from mohit is 0.9

Euclidean distance from Kunal is 1.2488788
Euclidean distance from mohit is 0.6433125
Euclidean distance from mohit_1 is 0.8733486
Euclidean distance from Kunal is 1.2580009
Euclidean distance from mohit is 0.5665213
Euclidean distance from mohit_1 is 0.80526423
Euclidean distance from Kunal is 1.2180642
Euclidean distance from mohit is 0.6178678
Euclidean distance from mohit_1 is 0.8484431
Euclidean distance from Kunal is 1.2366809
Euclidean distance from mohit is 0.67869526
Euclidean distance from mohit_1 is 0.9089612
Euclidean distance from Kunal is 1.1738733
Euclidean distance from mohit is 0.81925064
Euclidean distance from mohit_1 is 1.0078032
Euclidean distance from Kunal is 1.2469791
Euclidean distance from mohit is 0.599146
Euclidean distance from mohit_1 is 0.8162246
Euclidean distance from Kunal is 1.2656037
Euclidean distance from mohit is 0.6128887
Euclidean distance from mohit_1 is 0.8405221
Euclidean distance from Kunal is 1.334305
Euclidean distance from mohit is 0.5105

Euclidean distance from Kunal is 1.0033305
Euclidean distance from mohit is 0.6589715
Euclidean distance from mohit_1 is 0.8284035
Euclidean distance from Kunal is 1.062524
Euclidean distance from mohit is 0.7814319
Euclidean distance from mohit_1 is 0.97984856
Euclidean distance from Kunal is 1.1137764
Euclidean distance from mohit is 0.66483915
Euclidean distance from mohit_1 is 0.8319866
Euclidean distance from Kunal is 1.0054414
Euclidean distance from mohit is 0.7948538
Euclidean distance from mohit_1 is 0.97305745
Euclidean distance from Kunal is 1.1346377
Euclidean distance from mohit is 0.6824928
Euclidean distance from mohit_1 is 0.8661463
Euclidean distance from Kunal is 1.1993747
Euclidean distance from mohit is 0.6932853
Euclidean distance from mohit_1 is 0.91875494
Euclidean distance from Kunal is 1.2200671
Euclidean distance from mohit is 0.63348114
Euclidean distance from mohit_1 is 0.81173825
Euclidean distance from Kunal is 1.1548917
Euclidean distance from mohit is 0.

Euclidean distance from mohit_1 is 0.75035954
Euclidean distance from Kunal is 1.1669226
Euclidean distance from mohit is 0.60626274
Euclidean distance from mohit_1 is 0.805411
Euclidean distance from Kunal is 1.2324035
Euclidean distance from mohit is 0.5749649
Euclidean distance from mohit_1 is 0.7849584
Euclidean distance from Kunal is 1.1198307
Euclidean distance from mohit is 0.6906878
Euclidean distance from mohit_1 is 0.89390737
Euclidean distance from Kunal is 1.2267405
Euclidean distance from mohit is 0.7035402
Euclidean distance from mohit_1 is 0.9321607
Euclidean distance from Kunal is 1.2327595
Euclidean distance from mohit is 0.5804455
Euclidean distance from mohit_1 is 0.7585706
Euclidean distance from Kunal is 1.2286729
Euclidean distance from mohit is 0.5803567
Euclidean distance from mohit_1 is 0.7829475
Euclidean distance from Kunal is 1.1721197
Euclidean distance from mohit is 0.61213076
Euclidean distance from mohit_1 is 0.81158215
Euclidean distance from Kunal is 1

...

Euclidean distance from Kunal is 1.2135206
Euclidean distance from mohit is 0.50246406
Euclidean distance from mohit_1 is 0.66838825
Euclidean distance from Kunal is 1.1221561
Euclidean distance from mohit is 0.7422279
Euclidean distance from mohit_1 is 0.9521585
Euclidean distance from Kunal is 1.189642
Euclidean distance from mohit is 0.8172874
Euclidean distance from mohit_1 is 1.0224066
Euclidean distance from Kunal is 1.196564
Euclidean distance from mohit is 0.4934414
Euclidean distance from mohit_1 is 0.6029857
Euclidean distance from Kunal is 1.2587053
Euclidean distance from mohit is 0.59741575
Euclidean distance from mohit_1 is 0.7918435
Euclidean distance from Kunal is 1.307848
Euclidean distance from mohit is 0.69843245
Euclidean distance from mohit_1 is 0.92146426
Euclidean distance from Kunal is 1.3163232
Euclidean distance from mohit is 0.59814733
Euclidean distance from mohit_1 is 0.7839976
Euclidean distance from Kunal is 1.306157
Euclidean distance from mohit is 0.557

Euclidean distance from Kunal is 1.1002879
Euclidean distance from mohit is 0.6255208
Euclidean distance from mohit_1 is 0.74529654
Euclidean distance from Kunal is 1.2351965
Euclidean distance from mohit is 0.4863956
Euclidean distance from mohit_1 is 0.6304933
Euclidean distance from Kunal is 1.3055664
Euclidean distance from mohit is 0.58217865
Euclidean distance from mohit_1 is 0.801913
Euclidean distance from Kunal is 1.2971604
Euclidean distance from mohit is 0.5686495
Euclidean distance from mohit_1 is 0.7487489
Euclidean distance from Kunal is 1.2587577
Euclidean distance from mohit is 0.5568316
Euclidean distance from mohit_1 is 0.7287404
Euclidean distance from Kunal is 1.1705135
Euclidean distance from mohit is 0.6007332
Euclidean distance from mohit_1 is 0.80493313
Euclidean distance from Kunal is 1.2107892
Euclidean distance from mohit is 0.48988897
Euclidean distance from mohit_1 is 0.6703248
Euclidean distance from Kunal is 1.2141308
Euclidean distance from mohit is 0.63

Euclidean distance from Kunal is 1.2942936
Euclidean distance from mohit is 1.0369724
Euclidean distance from mohit_1 is 1.1442848
Euclidean distance from Kunal is 1.1699387
Euclidean distance from mohit is 0.87321335
Euclidean distance from mohit_1 is 0.98858243
Euclidean distance from Kunal is 1.3370348
Euclidean distance from mohit is 0.9797455
Euclidean distance from mohit_1 is 1.1246682
Euclidean distance from Kunal is 1.2745898
Euclidean distance from mohit is 0.56634784
Euclidean distance from mohit_1 is 0.74457055
Euclidean distance from Kunal is 1.2546518
Euclidean distance from mohit is 0.6186702
Euclidean distance from mohit_1 is 0.8291261
Euclidean distance from Kunal is 1.1922784
Euclidean distance from mohit is 0.6831601
Euclidean distance from mohit_1 is 0.9175056
Euclidean distance from Kunal is 1.128344
Euclidean distance from mohit is 0.7365306
Euclidean distance from mohit_1 is 0.9140487
Euclidean distance from Kunal is 1.1224794
Euclidean distance from mohit is 0.79

Euclidean distance from Kunal is 1.1784489
Euclidean distance from mohit is 0.7107137
Euclidean distance from mohit_1 is 0.93454933
Euclidean distance from Kunal is 1.1959517
Euclidean distance from mohit is 0.7525884
Euclidean distance from mohit_1 is 0.98677063


KeyboardInterrupt: 